In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf #

from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

from tkinter import *
from PIL import ImageTk, Image, ImageDraw

In [2]:
train = pd.read_csv('data/digit-recognizer/train.csv')
test = pd.read_csv('data/digit-recognizer/test.csv')

In [3]:
train_y = train['label']
train_X = train.drop('label',axis = 1)

In [4]:
ttr = np.array(train_X, dtype=np.float32)/255
ttr = ttr.reshape(-1,28,28,1)

In [5]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=23,
                  width_shift_range=0.20,
                  shear_range=15,
                  zoom_range=0.2,
                  validation_split=0.25,
                  horizontal_flip=False)


In [6]:
generator_train  = datagen.flow(ttr,train_y,batch_size=256,subset='training')
generator_validation  = datagen.flow(ttr,train_y,batch_size=64,subset='validation')

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='digit-recognizer-model.hdf5',monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=1)

In [20]:
model_hist = model.fit(generator_train,validation_data=generator_validation,epochs=18) # uncomment if u want to train model by your self

Epoch 1/18
124/124 [==============================] - 289s 2s/step - loss: 1.0442 - accuracy: 0.6647 - val_loss: 0.2682 - val_accuracy: 0.9181
Epoch 2/18
124/124 [==============================] - 308s 2s/step - loss: 0.2522 - accuracy: 0.9236 - val_loss: 0.2136 - val_accuracy: 0.9333
Epoch 3/18
124/124 [==============================] - 296s 2s/step - loss: 0.1909 - accuracy: 0.9424 - val_loss: 0.1516 - val_accuracy: 0.9525
Epoch 4/18
124/124 [==============================] - 307s 2s/step - loss: 0.1416 - accuracy: 0.9567 - val_loss: 0.1338 - val_accuracy: 0.9586
Epoch 5/18
124/124 [==============================] - 355s 3s/step - loss: 0.1320 - accuracy: 0.9599 - val_loss: 0.1094 - val_accuracy: 0.9649
Epoch 6/18
124/124 [==============================] - 223s 2s/step - loss: 0.1116 - accuracy: 0.9653 - val_loss: 0.1157 - val_accuracy: 0.9644
Epoch 7/18
124/124 [==============================] - 215s 2s/step - loss: 0.1087 - accuracy: 0.9659 - val_loss: 0.1055 - val_accuracy: 0.9662

In [44]:
model.save('model.h5')

In [9]:
new_model = tf.keras.models.load_model('model.h5')

In [10]:
probability_model = tf.keras.Sequential([new_model, 
                                         tf.keras.layers.Softmax()])

In [12]:
class Paint(Frame):
 
    def __init__(self, parent):
        Frame.__init__(self, parent)
 
        self.parent = parent
        self.color = "black"
        self.brush_size = 8
        self.setUI()
 
    def draw(self, event):
        self.canv.create_oval(event.x - self.brush_size, event.y - self.brush_size, event.x + self.brush_size, event.y + self.brush_size,
                              fill=self.color, outline=self.color)
    def Sterka(self):
        self.canv.create_rectangle(0,0,281,281,fill="white")
      #  if (self.color == "black"):         
      #      self.color = 'white'         
      #      self.brush_size = 10
      #  elif self.color == "white":
      #      self.color = 'black'
      #      self.brush_size = 10
    
    def Prediction(self):
        self.canv.postscript(file="my_dram.ps", colormode="color")
        image = Image.open('my_dram.ps')
        image = Image.open('my_dram.ps')
        npr = np.asarray(image.resize((28, 28)).convert('L')).reshape(1,28,28,1)
        def to255(number):
            return abs(number-255)
        to255_vecotorize = np.vectorize(to255)
        npr_255 = to255_vecotorize(npr) / 255.0
        predictions = probability_model.predict(npr_255)
        print(predictions)
        return (predictions.argmax())
 
    def setUI(self):
        self.pack(fill=BOTH, expand=1)
        self.canv = Canvas(self,width=280, height=280, bg="white")
        self.columnconfigure(0, weight=1)
        self.rowconfigure(0, weight=1)
        self.canv.grid(padx=5, pady=5, sticky=E + W + S + N)
        self.canv.bind("<B1-Motion>", self.draw)
        global e1
        e1 = Entry(self, width=10)
        e1.grid(row=0, column=3)
        Save_btn = Button(self, text="Prediction", width=10 ,command=lambda: self.clear_insert())
        Save_btn.grid(row=0, column=1)
        Sterka_btn = Button(self, text="Sterka", width=10 ,command=lambda: self.Sterka())
        Sterka_btn.grid(row=0, column=2)

    def clear_insert(self):
        e1.delete(0, 10)
        e1.insert(0 , self.Prediction())
 


root = Tk()
app = Paint(root)
root.mainloop()

[[0.08543097 0.08543037 0.08606317 0.08543909 0.08543037 0.08543192
  0.08543099 0.08543116 0.2304806  0.08543134]]
[[0.0854886  0.0854886  0.08554897 0.22954778 0.0854886  0.08548861
  0.0854886  0.08647652 0.085494   0.08548972]]
[[0.08564023 0.08564026 0.08617074 0.22711267 0.08564023 0.08564024
  0.08564023 0.08658611 0.08627732 0.08565197]]
[[0.08564023 0.08564026 0.08617074 0.22711267 0.08564023 0.08564024
  0.08564023 0.08658611 0.08627732 0.08565197]]
[[0.0853374  0.0853374  0.0853374  0.08533741 0.23195896 0.08533741
  0.08533765 0.0853374  0.0853416  0.0853374 ]]
[[0.08613107 0.08613107 0.08632524 0.21882933 0.08613107 0.08613212
  0.08613107 0.08638547 0.09165575 0.08614784]]
[[0.08540285 0.08540285 0.08555651 0.23092069 0.08540285 0.08540285
  0.08540285 0.0856828  0.08542231 0.08540343]]
[[0.08533686 0.08533686 0.08533687 0.23196742 0.08533686 0.08533766
  0.08533686 0.08533686 0.08533686 0.08533686]]
[[0.08533686 0.08533686 0.08533687 0.23196742 0.08533686 0.08533766
  0.